In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob as tb
from textblob import Word
from nltk.corpus import stopwords
import neattext.functions as nfx
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from PIL import Image
plt.style.use('ggplot')
import string
from string import digits

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer

from afinn import Afinn
import warnings
warnings.simplefilter('ignore')

In [2]:
march_c1=pd.read_csv('MarchCluster1.csv')

In [3]:
march_c1.head(10)

,Unnamed: 0,features,score
0,0,covid,0.043651
1,1,corona,0.030520
2,2,case,0.017973
3,3,curfew,0.017002
4,4,people,0.014467
5,5,virus,0.013901
6,6,positive,0.013539
7,7,coronavirus,0.013247
8,8,test,0.012868
9,9,time,0.012452


In [4]:
w=march_c1['features'][5]
w

'virus'

In [5]:
s1='death'

In [6]:
res = tb(s1)

In [7]:
print(res.sentiment.polarity)

0.0


In [8]:
sid_obj= SentimentIntensityAnalyzer()

In [9]:
print(sid_obj.polarity_scores(s1)) 

{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.5994}


In [10]:
s='disgusting'

In [11]:
afn = Afinn()

In [12]:
score=afn.score(s)

In [13]:
score

-3.0

In [14]:
sentiment = ['positive' if score > 0
                          else 'negative' if score < 0
                              else 'neutral']

In [15]:
sentiment

['negative']

# --------- Main code ----------

In [207]:
march=pd.read_csv('MarchClusterTweet.csv')

In [208]:
march.shape

(2451, 10)

In [209]:
march.dropna(inplace=True)

In [210]:
march.shape

(2448, 10)

In [211]:
march.reset_index(inplace = True, drop = True)

In [212]:
march["score"]=np.nan
march['sentiment']=np.nan

In [246]:
negative_word=[['covid',-3],['corona',-3],['case',-2],['coronaviru',-3],['viru',-2],['pandem',-2],['disease',-2],['variant',-1],['help',-2],['posit',-1],['rise',-2],['acitv',-1],['wave',-1],['death',-3],['die',-3],['sad',-2]]
N_df= pd.DataFrame(negative_word, columns = ['words', 'scores'],)

In [269]:
positive_word=[['vaccin',+3],['dose',+1],['lockdown',+3],['mask',+1],['curfew',+2],['fight',+2],['remdesivir',+2],['covisheld',+2],['cowin',+2],['happy'+2]]
P_df= pd.DataFrame(positive_word, columns = ['words', 'scores'])

TypeError: can only concatenate str (not "int") to str

In [270]:
N_df

,words,scores
0,covid,-3
1,corona,-3
2,case,-2
3,coronaviru,-3
4,viru,-2
5,pandem,-2
6,disease,-2
7,variant,-1
8,help,-2
9,posit,-1


In [271]:
N_df.words.tolist()

['covid',
 'corona',
 'case',
 'coronaviru',
 'viru',
 'pandem',
 'disease',
 'variant',
 'help',
 'posit',
 'rise',
 'acitv',
 'wave',
 'death',
 'die',
 'sad']

In [272]:
if 'viru' in N_df.words.tolist():
    print(N_df.words.tolist().index('viru'))

4


In [273]:
# afn = Afinn()
# i=0
# for s in march['clean_tweet']:
#     s=march['clean_tweet'][i]
#     if s!=np.nan:
#         score=afn.score(s)
#         march['score'][i]=score
#         sentiment = ['positive' if score > 0
#                                 else 'negative' if score < 0
#                                     else 'neutral']
#         march['sentiment'][i]=sentiment
#         i+=1
#     else:
#         continue

In [274]:
def Convert_list(string):
    li = list(string.split(" "))
    return li
def listToString(s): 
    str1 = " " 
    return (str1.join(s))

In [275]:
def sentiment_Analyser(str):
    score_ls=[]
    afn = Afinn()
    ls=Convert_list(str)
    for word in ls:
        if word in N_df.words.tolist():
            i=N_df.words.tolist().index(word)
            score=N_df['scores'][i]
        elif word in P_df.words.tolist():
            i=P_df.words.tolist().index(word)
            score=P_df['scores'][i]
        else:
            score=afn.score(w)
        score_ls.append(score)
    return score_ls

In [276]:
sc=sentiment_Analyser(march['clean_tweet'][1])

In [277]:
march['clean_tweet'][1]

'pfizer moderna claim vaccin safe effect estim death asso pfizer vaccin spain german pfizer exempt legal suit politician collus chanc die covid vaccin '

In [278]:
march['sentiment'][1]

'neutral'

In [279]:
def sum_of_list(l):
  total = 0
  for val in l:
    total = total + val
  return total

In [280]:
def sentiment_scorer(str):
    sc=sentiment_Analyser(str)
    score=sum_of_list(sc)
    return score

In [281]:
sentiment_scorer(march['clean_tweet'][1])

0.0

In [282]:
i=0
for s in march['clean_tweet']:
    s=march['clean_tweet'][i]
    if s!=np.nan:
        score=sentiment_scorer(s)
        march['score'][i]=score
        if score >0:
            sentiment='positive'
        elif score <0:
            sentiment='negative'
        else:
            sentiment='neutral'
        # sentiment = ['positive' if score > 0
        #                         else 'negative' if score < 0
        #                             else 'neutral']
        march['sentiment'][i]=sentiment
        i+=1
    else:
        continue

In [283]:
march['sentiment'].value_counts()

negative    1463
neutral      589
positive     396
Name: sentiment, dtype: int64

In [284]:
ps_df=march[march['sentiment']=='positive']

In [285]:
ps_df.reset_index(inplace = True, drop = True)

In [290]:
ps_df['tweet'][80]

'The cumulative number of #COVID19 vaccine doses administered in the country crossed 2.80 crores (2,80,05,817) on March 12, says Union Health Ministry'